In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [4]:
# 1. 데이터 준비
#-------------------------------------------------------------------------------
# MNIST 데이터셋을 위한 변환(transform) 정의
# 이미지를 Tensor로 변환하고, 정규화(normalize)합니다.
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

# 학습용 데이터셋 로드
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

# 테스트용 데이터셋 로드
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

classes = tuple(str(i) for i in range(10))

100.0%
100.0%
100.0%
100.0%


In [5]:
# 2. 신경망 모델 정의
#-------------------------------------------------------------------------------
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = torch.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = torch.log_softmax(x, dim=1)
        return output

net = Net()

In [6]:
# 3. 손실 함수 및 옵티마이저 정의
#-------------------------------------------------------------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)


In [7]:
# 4. 모델 학습
#-------------------------------------------------------------------------------
print("학습을 시작합니다...")
for epoch in range(10):  # 10 에포크 동안 학습합니다.

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
        inputs, labels = data

        # 변화도(gradient) 매개변수를 0으로 만듭니다.
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화를 한 후
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 통계를 출력합니다.
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print('학습 완료')

학습을 시작합니다...
[1,   200] loss: 0.457
[1,   400] loss: 0.166
[1,   600] loss: 0.134
[1,   800] loss: 0.128
[2,   200] loss: 0.091
[2,   400] loss: 0.090
[2,   600] loss: 0.083
[2,   800] loss: 0.072
[3,   200] loss: 0.069
[3,   400] loss: 0.067
[3,   600] loss: 0.057
[3,   800] loss: 0.065
[4,   200] loss: 0.052
[4,   400] loss: 0.052
[4,   600] loss: 0.054
[4,   800] loss: 0.056
[5,   200] loss: 0.043
[5,   400] loss: 0.052
[5,   600] loss: 0.045
[5,   800] loss: 0.044
[6,   200] loss: 0.039
[6,   400] loss: 0.039
[6,   600] loss: 0.032
[6,   800] loss: 0.039
[7,   200] loss: 0.032
[7,   400] loss: 0.032
[7,   600] loss: 0.032
[7,   800] loss: 0.037
[8,   200] loss: 0.026
[8,   400] loss: 0.032
[8,   600] loss: 0.033
[8,   800] loss: 0.031
[9,   200] loss: 0.021
[9,   400] loss: 0.029
[9,   600] loss: 0.033
[9,   800] loss: 0.028
[10,   200] loss: 0.021
[10,   400] loss: 0.025
[10,   600] loss: 0.030
[10,   800] loss: 0.025
학습 완료


In [8]:
# 5. 모델 저장
#-------------------------------------------------------------------------------
PATH = './mnist_net.pth'
torch.save(net.state_dict(), PATH)
print(f'모델이 {PATH}에 저장되었습니다.')

모델이 ./mnist_net.pth에 저장되었습니다.


In [9]:
# 6. 모델 평가
#-------------------------------------------------------------------------------
correct = 0
total = 0
# 학습 중이 아니므로, 출력에 대한 변화도를 계산할 필요가 없습니다.
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # 신경망에 이미지를 통과시켜 출력을 계산합니다.
        outputs = net(images)
        # 가장 높은 값(energy)를 갖는 분류(class)를 정답으로 선택합니다.
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'테스트 데이터셋에서의 정확도: {100 * correct // total} %')

# 각 분류(class)에 대한 정확도 출력
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # 각 분류별로 올바른 예측을 수집합니다.
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

# 각 분류별 정확도를 출력합니다.
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f"'{classname}' 분류의 정확도: {accuracy:.1f} %")

테스트 데이터셋에서의 정확도: 98 %
'0' 분류의 정확도: 99.2 %
'1' 분류의 정확도: 99.1 %
'2' 분류의 정확도: 99.2 %
'3' 분류의 정확도: 97.8 %
'4' 분류의 정확도: 98.1 %
'5' 분류의 정확도: 98.9 %
'6' 분류의 정확도: 98.4 %
'7' 분류의 정확도: 97.8 %
'8' 분류의 정확도: 98.9 %
'9' 분류의 정확도: 97.2 %
